## Detrend & Standardize

In [ ]:
! pip install pyts
from pyts.decomposition import SingularSpectrumAnalysis

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# -- format and save agricultural data
ratio_fmt = pd.read_csv(fname[0])

# -- loop for five variables
for j in range(5):
  vb0 = pd.read_csv(fname[j])
  # filter useless rows
  vb0 = ratio_fmt[['State','County']].merge(vb0, how='left')

  vb = vb0.drop(columns=['State', 'County'])
  vb_t = vb.copy().T
  # -- detrend 
  vb_ny = vb.to_numpy()

  for i in range(len(vb)):
    # i=0
    to_dt=vb_ny[i,:][~np.isnan(vb_ny[i])].reshape(1,-1)
    # Singular Spectrum Analysis
    ssa = SingularSpectrumAnalysis(window_size=10)
    vb_ssa = ssa.fit_transform(to_dt)
    back_value = to_dt[0] - vb_ssa[0]
    # find the index of non-NA rows
    slt_ind = vb_t[i].index[~vb_t[i].isnull()]
    vb_t.loc[slt_ind,i] = back_value

  # -- tranpose back to normal format
  vb_to_st = vb_t.copy().T
  
  # -- standardize
  vb_st=vb_to_st.sub(vb_to_st.mean(1), axis=0).div(vb_to_st.std(1), axis=0)
  vb_st[['State', 'County']] = vb0[['State', 'County']]
  
  # -- format the final rows to merge
  vb_to_m = vb_st.melt(id_vars=['State','County'])
  vb_to_m = vb_to_m.rename(columns={'variable': 'year', 'value': vname2[j]})

  vb_to_m = vb_to_m.dropna()
  vb_to_m["year"]= vb_to_m["year"].str[-4:]
  vb_to_m['year'] =vb_to_m['year'].astype(int)
  vb_one = vb_to_m.merge(county_code[['State',	'state',	'County',	'county']],on=['State','County']).drop_duplicates()

  oname[j] = vb_one.copy() 
  oname[j].to_csv(sname_dtst[j])
  


# RandomForest

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
# -- creat feature and target variables
feature = df[df.columns[7:]]
target = df['vrb']

In [ ]:
# -- create train/test sets
feat_tr, feat_te, targ_tr, targ_te = train_test_split(feature, target, 
                                                          test_size=0.2, random_state=333)

In [ ]:
# -- select the parameter(s) to tune and the values to try
tuned_parameters = [{"min_samples_leaf" : [5, 10, 50], 
                     "max_features":[5, 8, 11],
                     "max_depth":[40, 60, 80, 100]}]

In [ ]:
# -- perform Grid Search
rfc_tune = RandomForestRegressor()
cv_tune = GridSearchCV(rfc_tune, tuned_parameters, n_jobs=-1)
cv_tune.fit(feat_tr, targ_tr)

# -- print out the params with the highest "score"
print(cv_tune.best_params_)

{'max_depth': 100, 'max_features': 8, 'min_samples_leaf': 5}


In [ ]:
# -- suppress overfitting after using grid search
rfr = RandomForestRegressor(max_depth = 100, max_features=8, min_samples_leaf=5, n_estimators=50)
rfr.fit(feat_tr, targ_tr)

# -- calculate training and testing MSE
mse1_tr = mean_squared_error(targ_tr, rfr1.predict(feat_tr))
mse1_te = mean_squared_error(targ_te, rfr1.predict(feat_te))

print("MSE Training : {0}".format(mse_tr))
print("MSE Testing  : {0}".format(mse_te))
print("variance Training : {0}".format(targ_tr.var(ddof=0)))
print("variance Testing  : {0}".format(targ_te.var(ddof=0)))
print('r2 on training sets:', r2_score(targ_tr, rfr.predict(feat_tr)))
print('r2 on test sets:', r2_score(targ_te, rfr.predict(feat_te)))

In [ ]:
# -- put the feature importances into a DataFrame
imp = pd.DataFrame()
imp["name"] = feat_tr.columns
imp["importance"] = rfr.feature_importances_

# -- display all rows
pd.set_option("display.max_rows", 100)

# -- sort by importance
imp = imp.sort_values("importance", ascending=False)

# -- make a bar chart of top 5 most important features
ax = imp[:5].plot("name", "importance", kind="bar", figsize=[7, 7])
dum = ax.set_xlabel("")